# Introduction to Multi-Armed Bandits——05 Thompson Sampling[3]

## 参考资料

1. Russo D J, Van Roy B, Kazerouni A, et al. A tutorial on thompson sampling[J]. Foundations and Trends® in Machine Learning, 2018, 11(1): 1-96.

2. [ts_tutorial](https://github.com/iosband/ts_tutorial)

项目代码地址: [https://github.com/yijunquan-afk/bandit-learning.git](https://github.com/yijunquan-afk/bandit-learning.git)

到目前为止，我们所考虑的 `Bernoulli bandit` 和最短路径例子中的共轭性质有利于简单和计算效率高的贝叶斯推理。 计算效率制定模型时的一个重要考虑。 然而，很多情况下我们需要更复杂的模型，而精确的贝叶斯推理在计算上是难以实现的。 幸运的是，我们有一些准确的方法可以用来对后验分布进行近似采样。（**Approximations**）

在本节中，我们将讨论四种近似后验取样的方法。`Gibbs sampling`、`Langevin Monte Carlo`、从`Laplace approximation`中抽样和`bootstrap`。 作为一个例子，我们考虑在线最短路径问题的一个变体。

## 一、Binary Feedback

在之前最短路的基础上。假设图代表了 $M$ 阶的 `binomial bridge`，让每个 $\theta_e$ 独立分布，且是 gamma分布的，$\mathbb{E}[\theta_e] = 1$, $\mathbb{E}[\theta_e^2] = 1.5$，观察值根据如下公式产生：
$$
\begin{equation*}
y_t | \theta \sim \begin{cases}
1 \qquad & \text{with probability } \frac{1}{1 + \exp\left(\sum_{e \in x_t} \theta_e - M\right)} \\
0 \qquad & \text{otherwise.}
\end{cases}
\end{equation*}
$$

我们将奖励看作评级，$r_t = y_t$。例如，一个Internet路线推荐服务。每天，系统推荐一条路线 $x_t$ ，并收到司机的反馈 $y_t$ ，表示该路线是否可取。当实现的遍历时间 $\sum_{e \in x_t}$ 低于先前的期望值 $M$ ，反馈往往是积极的(1)，反之亦然(0)。

这个新的模型不具有在第四节中利用的共轭性质，也不适合于有效的精确贝叶斯推理。 不过，这个问题可以通过近似的方法来解决。 为了说明这一点，下图显示了三个近似版本的TS应用于`twenty-stage binomial bridge `的在线最短路问题的结果。这些算法利用了`Langevin Monte Carlo`、`Laplace`近似和`bootstrap`这三种方法。 作为比较的基线，还绘制了贪心算法的应用结果。

![image-20230126103058446](https://note-image-1307786938.cos.ap-beijing.myqcloud.com/typora/%20image-20230126103058446.png)

在本节中，我们让 $f_{t-1}$ 代表在历史为 $\mathbb{H}_{t-1} = ((x_1, y_1),\ldots,(x_{t-1},y_{t-1}))$ 的条件下 $\theta$ 的后验密度。TS算法通过从 $f_{t-1}$ 中抽取参数向量 $\hat{\theta}$ 并求解  $\hat{\theta}$  下的最优路径来生成动作 $x_t$ 。我们所描述的方法产生了一个样本 $\hat{\theta}$ ，其分布近似于后验 $\hat{f}_{t-1}$，当精确的后验抽样不可行时，可以近似地实现TS的功能。

## 二、Gibbs Sampling

可以阅读这篇文章：[机器学习中的MC、MCMC、Gibbs采样](https://blog.csdn.net/scott198510/article/details/124779155)

Gibbs抽样是一种通用马尔可夫链蒙特卡罗(MCMC)算法，用于从多元概率分布中提取近似样本。它产生了一个采样参数序列 $(\hat{\theta}^{n} : n =0,1,2, \ldots)$，形成了一个稳定分布的马尔可夫链 $f_{t-1}$ 。在合理的技术条件下，该马尔可夫链的极限分布为其平稳分布，$\hat{\theta}^n$ 的分布收敛于 $f_{t-1}$。

Gibbs抽样从最初的猜测 $\hat{\theta}^{0}$ 开始，迭代遍历$n=1,\ldots,N$，对于第n次扫描，该算法迭代分量 $k=1,\ldots,K$，对于每个 $k$ 生成一维边际分布:

$$f^{n,k}_{t-1}(\theta_k) \propto f_{t-1}((\hat{\theta}^n_1, \ldots, \hat{\theta}^n_{k-1}, \theta_k, \hat{\theta}^{n-1}_{k+1}, \ldots, \hat{\theta}^{n-1}_K))$$

按照 $\hat{\theta}^n_k \sim f^{n,k}_{t-1}$ 对第k个分量进行采样。经过 $N$ 次的扫描，向量 $\hat{\theta}^{N}$ 被认为是近似的后验样本。

Gibbs抽样适用于广泛的问题，即使在从 $f_{t-1}$ 抽样时，在计算上也往往是可行的。这是因为从一维分布中取样比较简单。 也就是说，对于复杂的问题，Gibbs抽样在计算上仍有一定要求。 例如我们的二元反馈的路径推荐问题。在这种情况下，很容易实现Gibbs抽样的一个版本，它可以在一分钟内产生一个接近于后验样本的结果。然而， 在数百个时间段内运行数千次模拟是相当耗时的，需要更有效的近似方法。

## 三、Laplace Approximation

我们现在讨论一种方法，用**高斯分布来逼近潜在的复杂后验分布**。然后，来自这个较简单的高斯分布的样本可以作为感兴趣的后验分布的近似样本。 Chapelle和Li 提出了[这种方法](https://proceedings.neurips.cc/paper/2011/hash/e53a0a2978c28872a4505bdb51db06dc-Abstract.html)，用广告点击率的逻辑回归模型来近似显示广告问题中的TS。

设 $g$ 表示 $\R^K$ 上的概率密度函数，我们希望从中抽样。如果 $g$ 是单峰的（unimodal），它的对数密度 $\ln(g(\phi))$ 在其模 $\overline{\phi}$ 上是严格凹的，$g(\phi)= e^{\ln(g(\phi))}$在 $\overline{\phi}$ 周围急剧地达到顶峰。考虑围绕其模进行全局近似是很自然的。对数密度的二阶泰勒近似给出如下：

$$\ln(g(\phi)) \approx \ln(g(\overline{\phi})) - \frac{1}{2} (\phi - \overline{\phi})^\top C (\phi - \overline{\phi}),$$

其中$$C = -\nabla^2 \ln(g(\overline{\phi})).$$

我们可以使用

$$\tilde{g}(\phi) \propto e^{-\frac{1}{2} (\phi - \overline{\phi})^\top C (\phi - \overline{\phi})}.$$

作为 $g$ 的近似值。这与具有均值 $\overline{\phi}$ 和协方差 $C^{-1}$ 的高斯分布密度成正比，因此

$$\tilde{g}(\phi) = \sqrt{|C/2\pi|} e^{-\frac{1}{2} (\phi - \overline{\phi})^\top C (\phi - \overline{\phi})}.$$

我们称之为 $g$ 的 `Laplace Approximation`.  由于有高效的算法来生成高斯分布的样本，这为从 $g$ 中近似取样提供了一种可行的方法。

作为一个例子，让我们考虑拉普拉斯近似在例子`binary feedback`中的应用。贝叶斯规则表明 $f_{t-1}$ 的后验密度 $θ$ 满足:

$$
f_{t-1}(\theta) \propto f_0(\theta) \prod_{\tau=1}^{t-1} \left(\frac{1}{1 + \exp\left(\sum_{e \in x_\tau} \theta_e - M\right)}\right)^{y_\tau}  \left(\frac{\exp\left(\sum_{e \in x_\tau} \theta_e - M\right)}{1 + \exp\left(\sum_{e \in x_\tau} \theta_e - M\right)}\right)^{1-y_\tau}
$$

模式 $\overline{\theta}$ 可以通过最大化 $f_{t-1}$ 来有效计算，$f_{t-1}$ 是对数凹的。然后从高斯分布中抽取一个近似的后验样本 $\hat{\theta}$，其平均值为 $\overline{\theta}$，协方差矩阵为 $(- \nabla^2 \ln(f_{t-1}(\overline{\theta})))^{-1}$

An approximate posterior sample $\hat{\theta}$ is then drawn from a Gaussian distribution with mean $\overline{\theta}$ and covariance matrix $(- \nabla^2 \ln(f_{t-1}(\overline{\theta})))^{-1}$.

拉普拉斯近似法非常适合于`binray feedback`，因为对数后验密度是严格内凹的，其梯度和Hessian可以有效计算。事实上，更广泛地说，拉普拉斯近似对具有平滑密度的后验分布是有效的，这些密度在其模式周围呈尖锐的峰值。当人们能够有效地计算后验模式，并且能够有效地形成对数后验密度的Hessian时，它们往往具有计算效率。
